In [2]:

import os
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import linregress, norm
import matplotlib.pyplot as plt

# -------------------------
# Utility: Mann-Kendall test
# -------------------------
def mann_kendall_test(x):
    """
    Compute Mann-Kendall test statistic and two-sided p-value.
    Returns: (S, Z, p_value)
    This implementation includes tie correction.
    """
    x = np.array(x)
    n = len(x)
    # remove nan
    mask = ~np.isnan(x)
    x = x[mask]
    n = len(x)
    if n < 3:
        return np.nan, np.nan, np.nan

    # S statistic
    S = 0
    for k in range(n - 1):
        S += np.sum(np.sign(x[k+1:] - x[k]))

    # tie corrections
    unique, counts = np.unique(x, return_counts=True)
    tie_sum = np.sum(counts * (counts - 1) * (2 * counts + 5))

    var_s = (n*(n-1)*(2*n+5) - tie_sum) / 18.0
    if var_s == 0:
        return S, np.nan, np.nan

    # Z statistic
    if S > 0:
        Z = (S - 1) / np.sqrt(var_s)
    elif S < 0:
        Z = (S + 1) / np.sqrt(var_s)
    else:
        Z = 0.0

    p = 2 * (1 - norm.cdf(abs(Z)))  # two-sided p-value
    return S, Z, p

# -------------------------
# Paths & Output directory
# -------------------------
INPUT_FILE = Path(r"D:\climate change\monthly_averages.xlsx")
OUTPUT_DIR = Path(r"D:\climate change\trend_results")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR = OUTPUT_DIR / "plots"
PLOTS_DIR.mkdir(exist_ok=True)
HEATMAPS_DIR = OUTPUT_DIR / "heatmaps"
HEATMAPS_DIR.mkdir(exist_ok=True)

# -------------------------
# Load data
# -------------------------
rain = pd.read_excel(INPUT_FILE, sheet_name="Rainfall_monthly", index_col=0).drop(["Lon", "Lat"], errors="ignore")
tmax = pd.read_excel(INPUT_FILE, sheet_name="Tmax_monthly", index_col=0).drop(["Lon", "Lat"], errors="ignore")
tmin = pd.read_excel(INPUT_FILE, sheet_name="Tmin_monthly", index_col=0).drop(["Lon", "Lat"], errors="ignore")

# convert index to datetime
rain.index = pd.to_datetime(rain.index)
tmax.index = pd.to_datetime(tmax.index)
tmin.index = pd.to_datetime(tmin.index)

# mean monthly temperature
temp = (tmax + tmin) / 2

# prepare numeric x axis: fractional year (use dayofyear/365)
times = temp.index
x_numeric = times.year + (times.dayofyear - 1) / 365.0  # fractional year

# month series for season masks
months = pd.Series(times.month, index=times)

# station list (assume temp, rain have same columns)
stations = list(temp.columns)

# prepare result storage
records = []

# matrices for heatmaps (stations x metrics)
# We'll store slopes for Temperature and Rainfall for annual/summer/winter
slope_temp = {"annual": [], "summer": [], "winter": []}
slope_rain = {"annual": [], "summer": [], "winter": []}

# significance flags (True if significant by linregress p<0.05 OR MK p<0.05)
sig_temp = {"annual": [], "summer": [], "winter": []}
sig_rain = {"annual": [], "summer": [], "winter": []}

# -------------------------
# Helper to run regression & MK on a timeseries
# -------------------------
def analyze_series(y, x_numeric):
    """
    y: 1D array of values (same length as x_numeric)
    returns dict with slope (per year), intercept, r2, p_value (linregress),
    mk_p (Mann-Kendall), mk_S, mk_Z
    """
    y = np.array(y, dtype=float)
    # require at least 3 non-nan points
    mask = ~np.isnan(y)
    if mask.sum() < 3:
        return {"slope": np.nan, "intercept": np.nan, "r2": np.nan, "p": np.nan,
                "mk_S": np.nan, "mk_Z": np.nan, "mk_p": np.nan}
    x_use = x_numeric[mask]
    y_use = y[mask]
    # linregress expects 1d arrays
    slope, intercept, r_value, p_value, stderr = linregress(x_use, y_use)
    r2 = r_value**2
    S, Z, mk_p = mann_kendall_test(y_use)
    return {"slope": slope, "intercept": intercept, "r2": r2, "p": p_value,
            "mk_S": S, "mk_Z": Z, "mk_p": mk_p}

# -------------------------
# Loop through stations
# -------------------------
for station in stations:
    y_temp = temp[station].values
    y_rain = rain[station].values

    # annual analysis
    res_temp_ann = analyze_series(y_temp, x_numeric)
    res_rain_ann = analyze_series(y_rain, x_numeric)

    # seasonal masks
    summer_mask = months.isin([4,5,6,7,8,9])   # Apr-Sep
    winter_mask = months.isin([10,11,12,1,2,3]) # Oct-Mar

    # build numeric arrays for seasons (we'll pick the subset)
    x_numeric_arr = np.array(x_numeric)

    # summer analysis
    res_temp_summer = analyze_series(y_temp[summer_mask.values], x_numeric_arr[summer_mask.values])
    res_rain_summer = analyze_series(y_rain[summer_mask.values], x_numeric_arr[summer_mask.values])

    # winter analysis
    res_temp_winter = analyze_series(y_temp[winter_mask.values], x_numeric_arr[winter_mask.values])
    res_rain_winter = analyze_series(y_rain[winter_mask.values], x_numeric_arr[winter_mask.values])

    # significance criteria: p<0.05 or mk_p<0.05
    def signif(res):
        p = res.get("p", np.nan)
        mk_p = res.get("mk_p", np.nan)
        return (not np.isnan(p) and p < 0.05) or (not np.isnan(mk_p) and mk_p < 0.05)

    # append to slopes arrays
    for key, r in [("annual", res_temp_ann), ("summer", res_temp_summer), ("winter", res_temp_winter)]:
        slope_temp[key].append(r["slope"])
        sig_temp[key].append(signif(r))

    for key, r in [("annual", res_rain_ann), ("summer", res_rain_summer), ("winter", res_rain_winter)]:
        slope_rain[key].append(r["slope"])
        sig_rain[key].append(signif(r))

    # Save record for CSV summary
    records.append({
        "Station": station,
        # temp annual
        "Temp_slope_annual_C_per_year": res_temp_ann["slope"],
        "Temp_r2_annual": res_temp_ann["r2"],
        "Temp_p_annual": res_temp_ann["p"],
        "Temp_mk_p_annual": res_temp_ann["mk_p"],
        "Temp_significant_annual": signif(res_temp_ann),
        # temp summer
        "Temp_slope_summer_C_per_year": res_temp_summer["slope"],
        "Temp_p_summer": res_temp_summer["p"],
        "Temp_mk_p_summer": res_temp_summer["mk_p"],
        "Temp_significant_summer": signif(res_temp_summer),
        # temp winter
        "Temp_slope_winter_C_per_year": res_temp_winter["slope"],
        "Temp_p_winter": res_temp_winter["p"],
        "Temp_mk_p_winter": res_temp_winter["mk_p"],
        "Temp_significant_winter": signif(res_temp_winter),

        # rain annual
        "Rain_slope_annual_mm_per_year": res_rain_ann["slope"],
        "Rain_r2_annual": res_rain_ann["r2"],
        "Rain_p_annual": res_rain_ann["p"],
        "Rain_mk_p_annual": res_rain_ann["mk_p"],
        "Rain_significant_annual": signif(res_rain_ann),
        # rain summer
        "Rain_slope_summer_mm_per_year": res_rain_summer["slope"],
        "Rain_p_summer": res_rain_summer["p"],
        "Rain_mk_p_summer": res_rain_summer["mk_p"],
        "Rain_significant_summer": signif(res_rain_summer),
        # rain winter
        "Rain_slope_winter_mm_per_year": res_rain_winter["slope"],
        "Rain_p_winter": res_rain_winter["p"],
        "Rain_mk_p_winter": res_rain_winter["mk_p"],
        "Rain_significant_winter": signif(res_rain_winter),
    })

    # -------------------------
    # Plot trend lines for station (annual)
    # -------------------------
    try:
        fig, ax = plt.subplots(figsize=(8, 4))
        # Temperature time-series and regression line
        ax.plot(times, y_temp, label="Temp (mean monthly)")
        # regression line values
        if not np.isnan(res_temp_ann["slope"]):
            line_y = res_temp_ann["intercept"] + res_temp_ann["slope"] * x_numeric
            ax.plot(times, line_y, linestyle='--', label=f"Temp trend: {res_temp_ann['slope']:.4f} °C/yr (p={res_temp_ann['p']:.3f})")
        ax.set_title(f"{station} — Temperature trend (annual)")
        ax.set_xlabel("Time")
        ax.set_ylabel("Temperature (°C)")
        ax.legend(fontsize="small")
        plt.tight_layout()
        fig_path = PLOTS_DIR / f"{station}_temp_annual_trend.png"
        fig.savefig(fig_path)
        plt.close(fig)
    except Exception as e:
        print(f"Warning plotting temp for {station}: {e}")

    try:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(times, y_rain, label="Rainfall (monthly)")
        if not np.isnan(res_rain_ann["slope"]):
            line_y = res_rain_ann["intercept"] + res_rain_ann["slope"] * x_numeric
            ax.plot(times, line_y, linestyle='--', label=f"Rain trend: {res_rain_ann['slope']:.4f} mm/yr (p={res_rain_ann['p']:.3f})")
        ax.set_title(f"{station} — Rainfall trend (annual)")
        ax.set_xlabel("Time")
        ax.set_ylabel("Rainfall (mm)")
        ax.legend(fontsize="small")
        plt.tight_layout()
        fig_path = PLOTS_DIR / f"{station}_rain_annual_trend.png"
        fig.savefig(fig_path)
        plt.close(fig)
    except Exception as e:
        print(f"Warning plotting rain for {station}: {e}")

# -------------------------
# Save CSV summary
# -------------------------
df_summary = pd.DataFrame.from_records(records)
summary_csv = OUTPUT_DIR / "trend_summary_per_station.csv"
df_summary.to_csv(summary_csv, index=False)
print(f"Saved summary CSV: {summary_csv}")

# -------------------------
# Heatmaps: prepare matrices
# -------------------------
labels = stations

def plot_heatmap(values_list, title, outpath, significance_mask=None, vmin=None, vmax=None):
    """
    values_list: list/array with len == n_stations
    significance_mask: list/array of booleans same length (True means significant)
    Saves heatmap image at outpath.
    """
    arr = np.array(values_list)[None, :]  # 2D for imshow (1 x n)
    fig, ax = plt.subplots(figsize=(max(8, len(labels)*0.3), 2.5))
    im = ax.imshow(arr, aspect="auto", vmin=vmin, vmax=vmax)

    # xticks = station names
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=90, fontsize=8)
    ax.set_yticks([0])
    ax.set_yticklabels([title], fontsize=9)
    ax.tick_params(axis='x', which='both', length=0)

    # add colorbar
    cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.02)
    cbar.ax.tick_params(labelsize=8)

    # overlay significance markers if provided
    if significance_mask is not None:
        for i, sig in enumerate(significance_mask):
            if sig:
                # draw a small triangle marker above the cell
                ax.text(i, 0, "▲", ha="center", va="bottom", fontsize=8)

    plt.tight_layout()
    fig.savefig(outpath, dpi=200)
    plt.close(fig)

# Compute reasonable vmin/vmax for slopes for visualization (using temperature slopes)
all_temp_slopes = np.hstack([slope_temp[k] for k in slope_temp])
vmin_temp, vmax_temp = np.nanpercentile(all_temp_slopes, [2, 98]) if np.isfinite(all_temp_slopes).any() else (None, None)
all_rain_slopes = np.hstack([slope_rain[k] for k in slope_rain])
vmin_rain, vmax_rain = np.nanpercentile(all_rain_slopes, [2,98]) if np.isfinite(all_rain_slopes).any() else (None, None)

# Temperature heatmaps
for key in ["annual", "summer", "winter"]:
    out = HEATMAPS_DIR / f"temp_slope_{key}.png"
    plot_heatmap(slope_temp[key], f"T slope ({key}) °C/yr", out, significance_mask=sig_temp[key], vmin=vmin_temp, vmax=vmax_temp)
    print(f"Saved heatmap: {out}")

# Rain heatmaps
for key in ["annual", "summer", "winter"]:
    out = HEATMAPS_DIR / f"rain_slope_{key}.png"
    plot_heatmap(slope_rain[key], f"Rain slope ({key}) mm/yr", out, significance_mask=sig_rain[key], vmin=vmin_rain, vmax=vmax_rain)
    print(f"Saved heatmap: {out}")

# -------------------------
# Create a short summary of significant stations
# -------------------------
sig_rows = []
for rec in records:
    st = rec["Station"]
    # check any significant for temp or rain across seasonal/annual
    temp_sig_any = rec["Temp_significant_annual"] or rec["Temp_significant_summer"] or rec["Temp_significant_winter"]
    rain_sig_any = rec["Rain_significant_annual"] or rec["Rain_significant_summer"] or rec["Rain_significant_winter"]
    sig_rows.append({"Station": st, "Temp_any_significant": temp_sig_any, "Rain_any_significant": rain_sig_any})

df_sig = pd.DataFrame(sig_rows)
sig_csv = OUTPUT_DIR / "significant_stations_summary.csv"
df_sig.to_csv(sig_csv, index=False)
print(f"Saved significant stations summary: {sig_csv}")

print("All plots & heatmaps saved to:", OUTPUT_DIR)
print("Per-station plots saved to:", PLOTS_DIR)


Saved summary CSV: D:\climate change\trend_results\trend_summary_per_station.csv
Saved heatmap: D:\climate change\trend_results\heatmaps\temp_slope_annual.png
Saved heatmap: D:\climate change\trend_results\heatmaps\temp_slope_summer.png
Saved heatmap: D:\climate change\trend_results\heatmaps\temp_slope_winter.png
Saved heatmap: D:\climate change\trend_results\heatmaps\rain_slope_annual.png
Saved heatmap: D:\climate change\trend_results\heatmaps\rain_slope_summer.png
Saved heatmap: D:\climate change\trend_results\heatmaps\rain_slope_winter.png
Saved significant stations summary: D:\climate change\trend_results\significant_stations_summary.csv
All plots & heatmaps saved to: D:\climate change\trend_results
Per-station plots saved to: D:\climate change\trend_results\plots
